In [1]:
from cassandra.cluster import Cluster
import pandas as pd
import numpy as np
from datetime import datetime

In [2]:
cluster = Cluster(['127.0.0.1'])
session = cluster.connect()
session.set_keyspace('data_stock')

In [3]:
rows_price = session.execute("""
    SELECT * FROM candlestick_data ALLOW FILTERING
""")
df_cand = pd.DataFrame(rows_price)
df_cand['time'] = pd.to_datetime(df_cand['time'])
df_cand = df_cand.sort_values(['symbol', 'time'])


In [4]:
rows_fin2 = session.execute("""
    SELECT symbol,eps,pe,pbv,percentYield FROM financal_data_fromsettradeAPI ALLOW FILTERING
""")
df_fun= pd.DataFrame(rows_fin2)
df_fun.head(10)

,symbol,eps,pe,pbv,percentyield
0,PPPM,0.02000,0.00,0.70,0.00
1,TPCH,0.24112,7.90,0.34,13.91
2,KPNREIT,NaN,NaN,0.23,0.00
3,POLY,0.27632,12.89,2.42,6.38
4,QHBREIT,NaN,NaN,0.41,0.00
5,VCOM,0.21428,7.85,1.48,10.79
6,KDH,4.12186,10.62,1.93,3.01
7,NVD,-0.00942,38.87,0.32,0.00
8,JDF,0.04224,12.19,1.35,4.85
9,SVR,-0.03402,0.00,0.34,0.00


In [5]:
candlestick_data=df_cand
candlestick_data["time"] = pd.to_datetime(candlestick_data["time"])
# ✅ ดึงแท่งเทียนล่าสุดของแต่ละ symbol
latest_candle = candlestick_data.sort_values("time").groupby("symbol").tail(1)
latest_candle.shape

(912, 8)

In [6]:
# ✅ 1) ดึงแท่งล่าสุดของแต่ละ symbol
df_latest_candle = (
    df_cand.sort_values("time")
    .groupby("symbol", as_index=False)
    .tail(1)  # หรือใช้ .last() ก็ได้หลัง sort แล้ว
)

# ✅ 2) Merge กับข้อมูล fundamental
df = pd.merge(df_fun, df_latest_candle, on="symbol", how="inner")

# ✅ 3) คำนวณ marketcap
df["marketcap"] = df["close_price"] * df["volume"]


In [7]:
df = df.drop(columns=['time', 'close_price',
       'high_price', 'low_price', 'open_price', 'value', 'volume',])

In [8]:
import ta
def compute_technical_grades(df_candle, band_pct=0.015):
    df_result = []

    for symbol, df_sym in df_candle.groupby('symbol'):
        df_sym = df_sym.sort_values('time').copy()
        df_sym['ema5'] = df_sym['close_price'].ewm(span=5, adjust=False).mean()
        df_sym['ema15'] = df_sym['close_price'].ewm(span=15, adjust=False).mean()
        df_sym['ema35'] = df_sym['close_price'].ewm(span=35, adjust=False).mean()
        df_sym['ema89'] = df_sym['close_price'].ewm(span=89, adjust=False).mean()
        df_sym['ema200'] = df_sym['close_price'].ewm(span=200, adjust=False).mean()
        df_sym['rsi'] = ta.momentum.RSIIndicator(close=df_sym['close_price'], window=14).rsi()

        # ใช้แท่งล่าสุดในการประเมิน
        row = df_sym.iloc[-1]
        try:
            c = row['close_price']
            r = row['rsi']
            e5, e15, e35, e89, e200 = row['ema5'], row['ema15'], row['ema35'], row['ema89'], row['ema200']

            if c >= e5 and r >= 70:
                grade = 'a'
            elif c >= e35 and e35 >= e89:
                grade = 'b'
            elif c >= e89 and (max([e5,e15,e35,e89]) - min([e5,e15,e35,e89])) / np.mean([e5,e15,e35,e89]) <= band_pct:
                grade = 'c'
            elif c < e89 and c < e200 and e89 < e200:
                grade = 'd'
            elif c < e5 < e15 < e35 < e89 < e200 and r <= 30:
                grade = 'e'
            else:
                grade = None
        except:
            grade = None

        df_result.append({'symbol': symbol, 'trend_grade': grade})

    return pd.DataFrame(df_result)

In [10]:
df_result = compute_technical_grades(df_cand)    

In [11]:
df_result.dropna(inplace=True)

In [12]:
# ✅ Clustering กลุ่ม A–E ด้วย KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

features = ['eps', 'pe', 'pbv', 'percentyield', 'marketcap']
X = df[features].dropna()
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

kmeans = KMeans(n_clusters=5, random_state=42, n_init='auto')
kmeans_labels = kmeans.fit_predict(X_scaled)
df.loc[X.index, 'group'] = kmeans_labels

group_map = {i: chr(65+i) for i in range(5)}
df['group'] = df['group'].map(group_map)

In [13]:
# ✅ 1) สมมติว่า df_graded มี: symbol | trend_grade
#         และ df มี:        symbol | group

# ✅ 2) Merge โดยใช้คอลัมน์ 'symbol'
df_merged = pd.merge(df, df_result[['symbol', 'trend_grade']], on='symbol', how='inner')

# ✅ 3) สร้างคอลัมน์ quadrant (เช่น Aa, Be, Dc, etc.)
df_merged['quadrant'] = df_merged['group'] + df_merged['trend_grade']

In [19]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import torch.nn as nn
import torch.optim as optim
from pytorch_tabnet.tab_model import TabNetClassifier

In [ ]:
# ✅ เตรียมข้อมูล
df = df_merged.copy()
features = ['eps', 'pe', 'pbv', 'percentyield', 'marketcap', 'ema5', 'ema15', 'ema35', 'ema89', 'ema200', 'rsi']
df = df.dropna(subset=features + ['quadrant'])

In [21]:
df_merged.head(10)

,symbol,eps,pe,pbv,percentyield,marketcap,group,trend_grade,quadrant
0,TPCH,0.24112,7.90,0.34,13.91,162340.635979,B,d,Bd
1,KPNREIT,NaN,NaN,0.23,0.00,255.999994,NaN,d,NaN
2,POLY,0.27632,12.89,2.42,6.38,35444.999027,B,d,Bd
3,QHBREIT,NaN,NaN,0.41,0.00,482635.448028,NaN,d,NaN
4,KDH,4.12186,10.62,1.93,3.01,102000.000000,A,d,Ad
5,NVD,-0.00942,38.87,0.32,0.00,16301.000000,A,d,Ad
6,JDF,0.04224,12.19,1.35,4.85,368.000007,B,d,Bd
7,SVR,-0.03402,0.00,0.34,0.00,27491.999209,A,d,Ad
8,SUSCO,0.11000,8.75,0.55,8.37,417893.841455,B,d,Bd
9,STP,0.92828,6.74,1.37,5.60,74480.001450,B,d,Bd
